In [1]:
# Code to evaluate EPMC annotation to the Manual Annotation
# Code to compare results from ML methods and EPMC annotation

# (c) EMBL-EBI, September 2019
#
# Started: 23 Septmember  2019
# Updated: 24 Septmember  2019

_author_ = 'Santosh Tirunagari'

import os
import pandas as pd
import glob
import json
import csv
import sys

import multiprocessing

import numpy as np
import re

from nltk.tokenize import wordpunct_tokenize

import requests
# from pprint import pprint
import pandas as pd

from collections import defaultdict, Counter
import time
from requests.compat import urljoin

from tqdm import tqdm

In [2]:
result_path = '/nfs/gns/literature/machine-learning/evaluation/300articles/ML-NER/PCSE/public_trained/'

from nltk.tokenize import wordpunct_tokenize, WordPunctTokenizer
from tqdm import tqdm
import csv
import pandas as pd

test_df = pd.read_csv('/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/test.csv', sep='\t', names = ['pmc_id','sentence', 'gt'])
dev_df = pd.read_csv('/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/dev.csv', sep='\t', names = ['pmc_id','sentence', 'gt'])
train_df = pd.read_csv('/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/train.csv', sep='\t', names = ['pmc_id','sentence', 'gt'])

In [7]:
from flair.models import TextClassifier
from flair.data import Sentence, Token
from flair.models import SequenceTagger

flair_models = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/'
# load the model you trained
# gene_model = SequenceTagger.load(flair_models+'multi_bio_ner_model_gene/v01/'+'best-model.pt')
# disease_model = SequenceTagger.load(flair_models+'multi_bio_ner_model_disease/'+'best-model.pt')
organisms_model = SequenceTagger.load(flair_models+'multi_bio_ner_model_organisms/v01/'+'best-model.pt')


2019-12-12 17:49:27,526 loading file /nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/multi_bio_ner_model_organisms/v01/best-model.pt


In [9]:
from nltk.tokenize import WordPunctTokenizer

text_temp = 'Brief inactivation of c-Myc is mouse not sufficient for sustained regression of c-Myc-induced tumours of pancreatic islets and skin epidermis'

sentence = ' '.join(wordpunct_tokenize(text_temp))
sentence = Sentence(sentence)
organisms_model.predict(sentence)

for token in sentence:
# print what you need (text and NER value)
    print(f"{token.text}\t{token.get_tag('ner').value}")

Brief	O
inactivation	O
of	O
c	O
-	O
Myc	O
is	O
mouse	B-Species
not	O
sufficient	O
for	O
sustained	O
regression	O
of	O
c	O
-	O
Myc	O
-	O
induced	O
tumours	O
of	O
pancreatic	O
islets	O
and	O
skin	O
epidermis	O


In [10]:
####### Organisms ###################

train_result_file_name = 'PCSE_public_train_OG_IOB.csv'
dev_result_file_name = 'PCSE_public_dev_OG_IOB.csv'

with open(result_path + train_result_file_name, 'w', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(train_df.iterrows(), total=train_df.shape[0]):
        text_temp = each_annotation['sentence'].encode('utf-8').decode('utf-8')

        sentence = ' '.join(wordpunct_tokenize(text_temp))
        sentence = Sentence(sentence)
        organisms_model.predict(sentence)

        for token in sentence:
        # print what you need (text and NER value)
            ner_value = token.get_tag('ner').value
            if  ner_value == 'B-Species':
                ner_value = 'B-OG'
                
            if  ner_value == 'I-Species':
                ner_value = 'I-OG'   
            
            public_writer.writerow([token.text, ner_value])
        public_writer.writerow('')
        
        
with open(result_path + dev_result_file_name, 'a', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        text_temp = each_annotation['sentence'].encode('utf-8').decode('utf-8')

        sentence = ' '.join(wordpunct_tokenize(text_temp))
        sentence = Sentence(sentence)
        organisms_model.predict(sentence)

        for token in sentence:
        # print what you need (text and NER value)
            ner_value = token.get_tag('ner').value
            if  ner_value == 'B-Species':
                ner_value = 'B-OG'
                
            if  ner_value == 'I-Species':
                ner_value = 'I-OG'   
            
            public_writer.writerow([token.text, ner_value])
        public_writer.writerow('')        

  2%|▏         | 1767/80014 [03:40<3:51:59,  5.62it/s]

2019-12-13 13:56:33,479 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-13 13:56:33,481 Ignore 1 sentence(s) with no tokens.


  2%|▏         | 1984/80014 [04:18<4:53:15,  4.43it/s]

2019-12-13 13:57:11,193 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-13 13:57:11,195 Ignore 1 sentence(s) with no tokens.


  4%|▎         | 2998/80014 [07:26<2:36:14,  8.22it/s] 

...

 80%|███████▉  | 12859/16108 [31:28<04:46, 11.34it/s]

2019-12-13 17:30:46,002 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-13 17:30:46,004 Ignore 1 sentence(s) with no tokens.


 96%|█████████▌| 15470/16108 [38:43<01:43,  6.14it/s]

2019-12-13 17:38:01,301 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-13 17:38:01,302 Ignore 1 sentence(s) with no tokens.


100%|██████████| 16108/16108 [40:51<00:00,  6.57it/s]


In [ ]:
import csv
data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/BC2GM-IOB/test.tsv'
targets = ['GENE']
                
def load_IOBdataset(data_path,targets):
    """
    load the IOB dataset, which is in csv format
    :param data_path: path to the csv file of IOB dataset
    :type data_path: str
    :param targets: a list of interest types
    :type targets: List[str]
    :return: list of labels of every sentence in dataset
    :rtype: List[List[str
    """
    X = []
    y = []

    X_sent = []
    y_sent = []
    with open(data_path, 'r') as f:
        csv_reader = csv.reader(f, delimiter='\t')
        for line in csv_reader:
            if line:
                token, tag = line[0], line[-1]
                X_sent.append(token)
                if targets:
                    if tag.split('-')[-1] in set(targets):
                        y_sent.append(tag)
                    else:
                        y_sent.append('O')
                else:
                    y_sent.append(tag)
            else:
                # we reach the end of a sentence
                if len(X_sent) > 0:
                    X.append(' '.join(X_sent))
                    y.append(y_sent)
                X_sent = []
                y_sent = []
    return X, y

BC2GM_test,y = load_IOBdataset(data_path,targets)

In [ ]:
result_path = '/nfs/gns/literature/machine-learning/Santosh/public_train_test'
result_file_name = 'BC2GM_test_IOB.csv'

from nltk.tokenize import wordpunct_tokenize, WordPunctTokenizer
from tqdm import tqdm
import csv
import pandas as pd

with open(result_path + result_file_name, 'a', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for each_sentence in tqdm(BC2GM_test):
        text_temp = each_sentence

#         sentence = ' '.join(text_temp.split(' '))
        sentence = Sentence(text_temp)
        gene_model.predict(sentence)

        for token in sentence:
        # print what you need (text and NER value)
            ner_value = token.get_tag('ner').value
#             if  ner_value == 'B-GP':
#                 ner_value = 'B-GENE'
                
#             if  ner_value == 'I-GP':
#                 ner_value = 'I-GENE'  
            
            public_writer.writerow([token.text, ner_value])
        public_writer.writerow('')